In [1]:
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import inception_resnet_v2
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import collections
from PIL import Image
import operator
import numpy as np
import argparse
import cv2
from keras.layers.convolutional import AtrousConvolution1D
from keras.utils import to_categorical
from keras.layers import Input, Dropout, BatchNormalization, Activation, Add, Conv2DTranspose, ZeroPadding1D
import keras
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, UpSampling1D
from keras.models import Model, Sequential
from keras.layers import Input, concatenate, LSTM, Dense, RepeatVector, Embedding, TimeDistributed
from keras.layers import LSTM, GRU, MaxPooling1D
from keras.layers import RepeatVector
from keras.layers import Dense, Reshape
from keras.layers.core import Activation, Dense, Dropout, Flatten, Lambda
from tensorflow.python.keras.layers import Layer
import matplotlib.pyplot as plt
import re
import numpy as np
from keras.models import load_model
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling2D
import keras.backend as K

import pandas as pd
import pycurl
import requests
import os

Using TensorFlow backend.


In [2]:
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import inception_resnet_v2
from keras.applications import ResNet50

In [3]:
from numpy import argmax
from keras import regularizers
from keras.regularizers import l2

In [4]:
import glob as glob

In [5]:
csvs = glob.glob('*.csv')

In [6]:
csvs

['68080.csv',
 '68056.csv',
 '56304.csv',
 '57433.csv',
 '55731.csv',
 '56034.csv',
 '55939.csv',
 '57439.csv',
 '54618.csv']

In [7]:
df =  pd.read_csv(csvs[0], sep=' *huh',header=None)
from sklearn import utils
val_cnt = 150

/home/aminov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [8]:
df = utils.shuffle(df)
df_train = df[:-val_cnt]
df_test = df[-val_cnt:]

for cs in csvs[1:]:
    buf = pd.read_csv(cs, sep=' *huh',header=None)
    buf = utils.shuffle(buf)
    df_train = df_train.append(buf[:-val_cnt], ignore_index=True)
    df_test = df_test.append(buf[-val_cnt:], ignore_index=True)

/home/aminov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [9]:
df_train = utils.shuffle(df_train)
df_test = utils.shuffle(df_test)

def clear_dataset(df_train):
    for i,val in df_train.iterrows():
        #--------0---------
        dir_ = str(val[0][:-1].strip())

        #--------3---------
        shit_len = len('/var/spool/data/www')
        buf = val[3][1:].strip()
        path = buf[shit_len:]
        name = path.split('/')[-1]
        img_path = dir_+'/'+name+'.jpg'

        #--------2--------- 
        name = val[2][1:-1].strip().lower()

        #------------------
        df_train.at[i, 0] = dir_
        df_train.at[i, 3] = img_path
        df_train.at[i, 2] = name
        
clear_dataset(df_train)
clear_dataset(df_test)

## -------------------------------------------------------------------------------------------

In [10]:
categories = sorted(df_train[0].unique())
label_to_onehot = dict((c, to_categorical(i, num_classes=len(categories))) for i, c in enumerate(categories))
label_to_int = dict((c, i) for i, c in enumerate(categories))
int_to_label = dict((i, c) for i, c in enumerate(categories))

In [11]:
alphabet = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789-,;.’!?:\'"/\\&|_@#$%°«^&ø*~№`+—-=<>()[]{}'

char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

In [12]:
maxlen = 400

def char_one_hot(data):
    for _ in range(maxlen-len(data)):
        data+=' '
    data = data.lower()
    integer_encoded = [char_to_int[char] if char in char_to_int else char_to_int[' '] for char in data ]

    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
        
    return integer_encoded

In [55]:
base_model = Xception(weights="imagenet")
for l in base_model.layers:
    l.trainable = True

img_x = base_model.layers[-2].output

In [32]:
def create_conv_model():
    
    inputs = Input(shape=(None,), name='sent_input', dtype='int64')
    x = Embedding(len(alphabet), 64, input_length=None)(inputs)
       
    frst_conv = Conv1D(32, 1, activation='relu', padding='same')(x)
    
    conv1 = Conv1D(64, 2, activation='relu', padding='same')(x)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(64, 2, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(128, 2, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(128, 2, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(256, 2, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(3, strides=1, padding='same')(conv1)   
    conv1 = Conv1D(512, 2, activation='relu', padding='same')(conv1)
    end = BatchNormalization()(conv1)
   
    avg = GlobalAveragePooling1D()(end)
    
    concat_x = Dense(256,activation='relu', W_regularizer=l2(0.001))(avg)#(concat_x)
    
    output = Dense(len(categories), activation='softmax')(concat_x)
   
    #model = Model(inputs=[base_model.input, inputs], outputs = output)
    model = Model(inputs = inputs, outputs = output)
    
    return model
    
model = create_conv_model()

/home/aminov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="relu", kernel_regularizer=<keras.reg...)`


In [33]:
from keras import optimizers

In [34]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=['accuracy'])

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sent_input (InputLayer)      (None, None)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 64)          7040      
_________________________________________________________________
conv1d_31 (Conv1D)           (None, None, 64)          8256      
_________________________________________________________________
batch_normalization_26 (Batc (None, None, 64)          256       
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, None, 64)          0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, None, 64)          8256      
_________________________________________________________________
batch_normalization_27 (Batc (None, None, 64)          256       
__________

In [1]:
BS = 32
def generator_data(df_train):
    while True:
        step = 0
        for b in range(int(len(df_train)/BS)):
            itr=0
            X = np.zeros((BS,maxlen))
            Y = np.zeros((BS,len(categories)))
            for i, val in df_train[step:step+BS].iterrows():
                try:
                    y = label_to_onehot[val[0]]

                    text_x = np.array([char_one_hot(val[2])])

                    X[itr]=text_x
                    Y[itr]=y

                    #yield ( text_x, np.array([y]))
                except Exception as ex:
                    pass
                itr+=1
                
            step+=BS
            yield (X,Y)
            


In [37]:
model.fit_generator(generator_data(df_train), 
                    steps_per_epoch=int(len(df_train)/BS),
                    validation_data=generator_data(df_test),
                    validation_steps=int(len(df_test)/BS),
                    epochs=17
                   )

Epoch 1/17
236/236 [==============================] - 33s 141ms/step - loss: 0.6192 - acc: 0.8922 - val_loss: 0.7232 - val_acc: 0.8539
Epoch 2/17
236/236 [==============================] - 29s 124ms/step - loss: 0.4882 - acc: 0.9300 - val_loss: 0.6225 - val_acc: 0.8986
Epoch 3/17
236/236 [==============================] - 29s 123ms/step - loss: 0.4019 - acc: 0.9630 - val_loss: 0.4791 - val_acc: 0.9456
Epoch 4/17
236/236 [==============================] - 29s 124ms/step - loss: 0.3402 - acc: 0.9839 - val_loss: 0.5285 - val_acc: 0.9167
Epoch 5/17
236/236 [==============================] - 29s 124ms/step - loss: 0.2960 - acc: 0.9915 - val_loss: 0.4905 - val_acc: 0.9287
Epoch 6/17
236/236 [==============================] - 29s 123ms/step - loss: 0.2619 - acc: 0.9945 - val_loss: 0.4771 - val_acc: 0.9220
Epoch 7/17
236/236 [==============================] - 29s 124ms/step - loss: 0.2333 - acc: 0.9962 - val_loss: 0.3633 - val_acc: 0.9502
Epoch 8/17
236/236 [==============================] - 2

In [38]:
model.save('model.hdf5')

In [40]:
model.evaluate_generator(generator_data(df_test),int(len(df_test)/BS))

[0.07902339065358752, 0.9952877050354367]